# Setup

In [ ]:
using WAV
include("../tools/plotting.jl");
include("../model/model.jl");
include("../model/gammatonegram.jl");
include("../model/time_helpers.jl");

In [ ]:
using GenWorldModels

In [ ]:
source_params, steps, gtg_params, obs_noise = include("../params/base.jl")
sr = 2000.0
gtg_params["dB_threshold"] = 0.0
wts, = gtg_weights(sr, gtg_params);

# Model

In [ ]:
@type AudioSource

### Embed a sound within a larger scene

In [ ]:
function embed_in_scene(scene_length, sr, wave, onset)
  n_samples = Int(floor(sr * scene_length))
  scene_wave = zeros(n_samples)
  sample_start = max(1, Int(floor(onset * sr)))
  sample_finish = min(sample_start + length(wave), length(scene_wave))
  scene_wave[sample_start:sample_finish-1] = wave[1:length(sample_start:sample_finish-1)]
  return scene_wave
end

### Generate white noise sound

In [ ]:
@gen function generate_single_noise(scene_length, steps, sr)
  onset ~ uniform(0, scene_length)
  duration ~ uniform(0.1, 1.0)
  amp ~ normal(10.0, 8.0)
  times, t, f = get_gp_spectrotemporal([onset, onset+duration], steps, sr)
  noise_wave = generate_noise(transpose(reshape(fill(amp, length(times)), (length(f), length(t)))), duration, steps, sr, 1e-6)
  return embed_in_scene(scene_length, sr, noise_wave, onset)
end;

### Generate tone with pitch

In [ ]:
@gen function generate_single_tone(scene_length, step_size, sr)
  step_size = step_size["t"]
  erb ~ uniform(0.4, 37.0)
  onset ~ uniform(0.0, scene_length)
  duration ~ uniform(0.1, 1.0)
  times = get_element_gp_times([onset, onset + duration], step_size)
  wave = generate_tone(fill(erb, length(times)), fill(50.0, length(times)), duration, step_size, sr, 1.0e-6)
  return embed_in_scene(scene_length, sr, wave, onset)
end;

### Generate sound (noise or tone)

In [ ]:
@gen function generate_single_sound(world, source::AudioSource)
    scene_length ~ lookup_or_generate(world[:args][:scene_length])
    steps ~ lookup_or_generate(world[:args][:steps])
    sr ~ lookup_or_generate(world[:args][:sr])

    is_noise ~ bernoulli(0.4)
    if is_noise
        wave = {*} ~ generate_single_noise(scene_length, steps, sr)
    else
        wave = {*} ~ generate_single_tone(scene_length, steps, sr)
    end
    return wave
end
#generate_sounds = Map(generate_single_sound);

### Generate scene with many sounds

In [ ]:
@gen (static) function _generate_scene(world, wts, gtg_params)
  n_tones ~ uniform_discrete(1, 4)
    
  scene_duration ~ lookup_or_generate(world[:args][:scene_length])
  audio_sr ~ lookup_or_generate(world[:args][:sr])

  waves ~ Map(lookup_or_generate)([world[:waves][AudioSource(i)] for i=1:n_tones])
  n_samples = Int(floor(scene_duration * audio_sr))
  scene_wave = reduce(+, waves; init=zeros(n_samples))
  scene_gram, = gammatonegram(scene_wave, wts, audio_sr, gtg_params)
  scene ~ noisy_matrix(scene_gram, 1.0)
  return scene_gram, scene_wave, waves
end;

generate_scene = UsingWorld(
    _generate_scene,
    :waves => generate_single_sound;
    world_args=(:scene_length, :steps, :sr),
    oupm_types=(AudioSource,)
)

In [ ]:
@load_generated_functions

# Trace visualization and playback

In [ ]:
function vis_and_write_wave(tr, title)
  duration, _, sr, = get_args(tr)
  gram, scene_wave, = get_retval(tr)
  wavwrite(scene_wave/maximum(abs.(scene_wave)), title, Fs=sr)
  plot_gtg(gram, duration, sr, 0, 100)
end

Some default arguments to `generate_scene`:

In [ ]:
scene_length, steps, sr = (2.0, steps, sr)
args = (scene_length, steps, sr, wts, gtg_params)

Generate and visualize a trace:

In [ ]:
tr = simulate(generate_scene, args);
vis_and_write_wave(tr, "simulated_scene.wav")

Playback (works on Mac, maybe not other platforms):

In [ ]:
; afplay simulated_scene.wav

# Inference

In [ ]:
function generic_no_num_change_inference_iter(tr)
  for j=1:tr[:kernel => :n_tones]
      tr,_ = mh(tr, select(:world => :waves => AudioSource(j)))
      if tr[:world => :waves => AudioSource(j) => :is_noise]
        tr,_ = mh(tr, select(:world => :waves => AudioSource(j) => :amp))
      else
        tr,_ = mh(tr, select(:world => :waves => AudioSource(j) => :erb))
      end
      tr,_ = mh(tr, select(:world => :waves => AudioSource(j) => :onset))
      tr,_ = mh(tr, select(:world => :waves => AudioSource(j) => :duration))
  end
  return tr
end

function generic_inference_iter(tr)
  tr = generic_no_num_change_inference_iter(tr)
  tr,_ = mh(tr, select(:kernel => :n_tones))
  return tr
end

function do_generic_inference(tr, iters)
  println("Hello")
  for i=1:iters
    tr = generic_inference_iter(tr)
  end
  return tr
end

In [ ]:
@gen function birth_death_proposal(tr)
  do_birth ~ bernoulli(0.5)
  if do_birth
    idx ~ uniform_discrete(1, tr[:kernel => :n_tones] + 1)
  else
    idx ~ uniform_discrete(1, tr[:kernel => :n_tones])
  end
end


@oupm_involution birth_death_inv (old_tr, fwd_prop_tr) to (new_tr, bwd_prop_tr) begin
  do_birth = @read(fwd_prop_tr[:do_birth], :disc)
  idx = @read(fwd_prop_tr[:idx], :disc)
  num = @read(old_tr[:kernel => :n_tones], :disc)
  if do_birth
    @birth(AudioSource, idx)
    @write(new_tr[:kernel => :n_tones], num + 1, :disc)
    @regenerate(:world => :waves => AudioSource(idx))
  else
    @death(AudioSource, idx)
    @write(new_tr[:kernel => :n_tones], num - 1, :disc)
    @save_for_reverse_regenerate(:world => :waves => AudioSource(idx))
  end
  @write(bwd_prop_tr[:do_birth], !do_birth, :disc)
  @write(bwd_prop_tr[:idx], idx, :disc)
end

birth_death_mh_kern = OUPMMHKernel(birth_death_proposal, (), birth_death_inv)

In [ ]:
function birth_death_inference_iter(tr)
  tr = generic_no_num_change_inference_iter(tr)
  tr,_ = mh(tr, birth_death_mh_kern)
  return tr
end
function do_birth_death_inference(tr, iters)
  for i=1:iters
    tr = birth_death_inference_iter(tr)
  end
  return tr
end

# Testing

Generate ground truth:

In [ ]:
ground_truth = simulate(generate_scene, args);
vis_and_write_wave(ground_truth, "ground_truth.wav")

In [ ]:
; afplay ground_truth.wav

Generate trace with constraints:

In [ ]:
tr, = generate(generate_scene, args, choicemap((:kernel => :scene, ground_truth[:kernel => :scene])));

Run inference:

In [ ]:
inferred_trace1 = do_generic_inference(tr, 100)
get_score(inferred_trace1)

In [ ]:
vis_and_write_wave(inferred_trace1, "inferred1.wav")

In [ ]:
; afplay ground_truth.wav

In [ ]:
#observations = choicemap((:kernel => :scene, tr[:kernel => :scene]))
#inferred_tr2, = generate(generate_scene, (2.0, steps, sr, wts, gtg_params), observations);

In [ ]:
inferred_tr2 = do_birth_death_inference(tr, 100);
get_score(inferred_tr2)

In [ ]:
vis_and_write_wave(inferred_tr2, "inferred2.wav")

In [ ]:
; afplay inferred.wav

# Auditory illusion setup

In [ ]:
function tones_with_noise(amp)
    cm = choicemap((:kernel => :n_tones) => 3,
              (:world => :waves => AudioSource(1) => :is_noise) => false,
              (:world => :waves => AudioSource(1) => :erb) => 10.0,
              (:world => :waves => AudioSource(1) => :onset) => 0.5,
              (:world => :waves => AudioSource(1) => :duration) => 0.3,
              (:world => :waves => AudioSource(2) => :is_noise) => false,
              (:world => :waves => AudioSource(2) => :erb) => 10.0,
              (:world => :waves => AudioSource(2) => :onset) => 1.1,
              (:world => :waves => AudioSource(2) => :duration) => 0.3,
              (:world => :waves => AudioSource(3) => :is_noise) => true,
              (:world => :waves => AudioSource(3) => :amp) => amp,
              (:world => :waves => AudioSource(3) => :onset) => 0.8,
              (:world => :waves => AudioSource(3) => :duration) => 0.3)
    tr, = generate(generate_scene, args, cm)
    return tr
end

In [ ]:
trr = tones_with_noise(10.0);

In [ ]:
vis_and_write_wave(trr, "trr.wav")

In [ ]:
; afplay trr.wav

In [ ]:
observations = choicemap((:kernel => :scene, trr[:kernel => :scene]))
inferred_tr,weight = generate(generate_scene, args, observations);
weight

In [ ]:
inferred_tr[:kernel => :scene] == trr[:kernel => :scene]

In [ ]:
inferred_tr = do_birth_death_inference(inferred_tr, 500);
get_score(inferred_tr)

In [ ]:
vis_and_write_wave(inferred_tr, "inferred.wav")

In [ ]:
get_submap(get_choices(inferred_tr), :world => :waves)

In [ ]:
; afplay inferred.wav

In [ ]:
inferred_tr2 = do_birth_death_inference(inferred_tr, 200);
get_score(inferred_tr)

In [ ]:
vis_and_write_wave(inferred_tr2, "inferred2.wav")

In [ ]:
@dist function uniform_from_list(list)
    idx = uniform_discrete(1, length(list))
    list[idx]
end

In [ ]:
uniform_from_list([:a, :b, :c])

In [ ]:
@gen function splitmerge_prop(tr)
    n_tones = tr[:kernel => :n_tones]
    do_split ~ bernoulli(0.5)
    if do_split
        solo_idx ~ uniform_discrete(1, n_tones)
        deuce_idx1 ~ uniform_discrete(1, n_tones + 1)
        deuce_idx2 ~ uniform_discrete(1, n_tones + 1)
        if deuce_idx1 != deuce_idx2
            ch = get_submap(get_choices(tr), :world => :waves => AudioSource(solo_idx))
            if !ch[:is_noise]
                erb1 ~ normal(ch[:erb], 2.)
                erb2 ~ normal(ch[:erb], 2.)
            else
                amp1 ~ normal(ch[:amp], 2.)
                amp2 ~ normal(ch[:amp], 2.)
            end
            # the split sounds go from ch[:onset] to ch[:onset] + dur1, and
            # startpoint to ch[:onset]+ch[:duration]-dur2 to ch[:onset]+ch[:duration]
            dur1 ~ uniform(1, ch[:duration])
            dur2 ~ uniform(1, ch[:duration])
        end
    else
        solo_idx ~ uniform_discrete(1, max(1, n_tones - 1))
        deuce_idx1 ~ uniform_discrete(1, n_tones)
        if (deuce_idx1 > n_tones)
          # if this happens, this is the backward step for an impossible forward move;
          # just escape the function quickly in this case
          deuce_idx2 ~ uniform_discrete(1, n_tones)
          return nothing
        end
        ch1 = get_submap(get_choices(tr), :world => :waves => AudioSource(deuce_idx1))
        compatible_indices = [
            idx for idx=1:n_tones
            if (
                tr[:world => :waves => AudioSource(idx) => :is_noise] == ch1[:is_noise] && 
                tr[:world => :waves => AudioSource(idx) => :onset] >= ch1[:onset]
            )
        ]
        deuce_idx2 ~ uniform_from_list(compatible_indices)
        ch2 = get_submap(get_choices(tr), :world => :waves => AudioSource(deuce_idx2))
        
        if deuce_idx1 != deuce_idx2
            if ch1[:is_noise]
                amp ~ normal((ch1[:amp] + ch2[:amp])/2, 2.)
            else
                erb ~ normal((ch1[:erb] + ch2[:erb])/2, 2.)
            end    
        end
    end
end

@oupm_involution splitmerge_inv (old_tr, fwd_prop_tr) to (new_tr, bwd_prop_tr) begin
    n_tones = @read(old_tr[:kernel => :n_tones], :disc)
    do_split = @read(fwd_prop_tr[:do_split], :disc)
    deuce_idx1 = @read(fwd_prop_tr[:deuce_idx1], :disc)
    deuce_idx2 = @read(fwd_prop_tr[:deuce_idx2], :disc)
    solo_idx = @read(fwd_prop_tr[:solo_idx], :disc)
    if deuce_idx1 != deuce_idx2
        if do_split
            @split(AudioSource, solo_idx, deuce_idx1, deuce_idx2)
            @write(new_tr[:kernel => :n_tones], n_tones + 1, :disc)

            o(x) = :world => :waves => AudioSource(solo_idx) => x
            n1(x) = :world => :waves => AudioSource(deuce_idx1) => x
            n2(x) = :world => :waves => AudioSource(deuce_idx2) => x

            # copy is noise
            @copy(old_tr[o(:is_noise)], new_tr[n1(:is_noise)])
            @copy(old_tr[o(:is_noise)], new_tr[n2(:is_noise)])

            # start and end times
            @copy(old_tr[o(:onset)], new_tr[n1(:onset)])
            @copy(fwd_prop_tr[:dur1], new_tr[n1(:duration)])
            @copy(fwd_prop_tr[:dur2], new_tr[n2(:duration)])

            old_ons = @read(old_tr[o(:onset)], :cont)
            old_dur = @read(old_tr[o(:duration)], :cont)
            dur2 = @read(fwd_prop_tr[:dur2], :cont)
            @write(new_tr[n2(:onset)], old_ons + old_dur - dur2, :cont)

            # amp/erb
            if @read(old_tr[o(:is_noise)], :disc)
                @copy(fwd_prop_tr[:amp1], new_tr[n1(:amp)])
                @copy(fwd_prop_tr[:amp2], new_tr[n2(:amp)])
                @copy(old_tr[o(:amp)], bwd_prop_tr[:amp])
            else
                @copy(fwd_prop_tr[:erb1], new_tr[n1(:erb)])
                @copy(fwd_prop_tr[:erb2], new_tr[n2(:erb)])
                @copy(old_tr[o(:erb)], bwd_prop_tr[:erb])
            end
        else
            @merge(AudioSource, solo_idx, deuce_idx1, deuce_idx2)
            @write(new_tr[:kernel => :n_tones], n_tones - 1, :disc)
            
            n(x) = :world => :waves => AudioSource(solo_idx) => x
            o1(x) = :world => :waves => AudioSource(deuce_idx1) => x
            o2(x) = :world => :waves => AudioSource(deuce_idx2) => x

            # is_noise
            @copy(old_tr[o1(:is_noise)], new_tr[n(:is_noise)])

            # onset & duration
            start1 = @read(old_tr[o1(:onset)], :cont)
            dur1 = @read(old_tr[o1(:duration)], :cont)
            start2 = @read(old_tr[o2(:onset)], :cont)
            dur2 = @read(old_tr[o2(:duration)], :cont)
            end2 = start2 + dur2
            full_dur = end2 - start1

            @copy(old_tr[o1(:onset)], new_tr[n(:onset)])
            @write(new_tr[n(:duration)], full_dur, :cont)

            @write(bwd_prop_tr[:dur1], dur1, :cont)
            @write(bwd_prop_tr[:dur2], dur2, :cont)

            if @read(old_tr[o1(:is_noise)], :disc)
                @copy(old_tr[o1(:amp)], bwd_prop_tr[:amp1])
                @copy(old_tr[o2(:amp)], bwd_prop_tr[:amp2])
                @copy(fwd_prop_tr[:amp], new_tr[n(:amp)])
            else
                @copy(old_tr[o1(:erb)], bwd_prop_tr[:erb])
                @copy(old_tr[o2(:erb)], bwd_prop_tr[:erb])
                @copy(fwd_prop_tr[:erb], new_tr[n(:erb)])
            end
        end
    end
    @write(bwd_prop_tr[:do_split], !do_split, :disc)
    @copy(fwd_prop_tr[:solo_idx], bwd_prop_tr[:solo_idx])
    @copy(fwd_prop_tr[:deuce_idx1], bwd_prop_tr[:deuce_idx1])
    @copy(fwd_prop_tr[:deuce_idx2], bwd_prop_tr[:deuce_idx2])
end
splitmerge_mh_kern = OUPMMHKernel(splitmerge_prop, (), splitmerge_inv)

In [ ]:
function split_merge_inference_iter(tr)
  tr = generic_no_num_change_inference_iter(tr)
  tr,_ = mh(tr, birth_death_mh_kern)
  tr = generic_no_num_change_inference_iter(tr)
  tr,_ = mh(tr, splitmerge_mh_kern)
  return tr
end
function do_split_merge_inference(tr, iters)
  for i=1:iters
    tr = split_merge_inference_iter(tr)
  end
  return tr
end

In [ ]:
observations = choicemap((:kernel => :scene, trr[:kernel => :scene]))
pre_infer_tr,weight = generate(generate_scene, args, observations);
weight

In [ ]:
inferred_sm_tr = do_split_merge_inference(pre_infer_tr, 100)
get_score(inferred_sm_tr)

In [ ]:
vis_and_write_wave(inferred_sm_tr, "inferred_sm.wav")